In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import utilities # codeTimer context manager.
import data_preparation # Load dataset and build required matrices.
import factorisation # WALS factorisation.
import recommender # Recommender system.

### Loading dataset and creating recommender system

In [ ]:
np.random.seed(17)

mov, rat = data_preparation.importDataset(0.3)
rec = recommender.recommenderSystem(mov, rat)
print("Prediction MAE: {}".format(rec.predictionError()))

In [ ]:
# The a pre-trained recommander system can be loaded with the following line.

# rec = utilities.loadRecSys("rec.pkl")

### Selecting best regression coefficient with CV

In [ ]:
np.random.seed(17)

reg_lambda = [0.10, 0.20, 0.30]
n_folds = 5
n_iter = 4

# reg_lambda is required to be a list.
with utilities.codeTimer("Best regression lambda CV"):
    best_lambda = rec.bestLambdaCV(n_folds, n_iter, reg_lambda)

### Factorisation

In [ ]:
# Expect ~ 24 min for the execution on the whole dataset.
#reg_lambda = best_lambda
reg_lambda = 0.2
n_iter = 5

with utilities.codeTimer("WALS factorisation"):
    train, test = rec.performFactorisation(reg_lambda, n_iter)

In [ ]:
# Once the recommender has been trained, it can be saved with the
# following line.

# utilities.saveRecSys(rec, "rec.pkl")

### Recommendation

In [ ]:
def recommend(rec_system, user_id):
    return rec_system.answerQuery(user_id)
        
def bestRated(rec_system, user_id):
    user_movies = rec_system.getUserMovies(user_id)
    return user_movies.sort_values(by = "Rating", ascending = False)

In [ ]:
user_id = 13
recommend(rec, user_id).head(10)

In [ ]:
bestRated(rec, user_id).head(10)

### Similar items
Some suggestions:
* 911: Star Wars Episode VI
* 786: Dumbo
* 957: The Shining
* 474: Blade Runner

In [ ]:
rec.suggestSimilar(957)

### New user recommendation

In [ ]:
np.random.seed(17)

new_user, new_user_id = rec.generateNewUser(130)
np.shape(rec.R)

In [ ]:
new_user_id

In [ ]:
with utilities.codeTimer("New user factorisation"):
    rec.addNewUser(new_user, reg_lambda)
np.shape(rec.R)

In [ ]:
recommend(rec, new_user_id).head(10)

In [ ]:
bestRated(rec, new_user_id).head(10)

### Cold start problem
If a new user has rated less than 10 movies, the most popular and unseen movies will be recommended.

In [ ]:
np.random.seed(17)

new_user, new_user_id = rec.generateNewUser(8)
np.shape(rec.R)

with utilities.codeTimer("New user factorisation"):
    rec.addNewUser(new_user, reg_lambda)

In [ ]:
recommend(rec, new_user_id).head(10)